In [1]:
import openml, fairlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer
from keras.models import Sequential
from keras.layers import Dense
from fairlib.inprocessing import Fauci

INFO:fairlib:fairlib loaded
INFO:fairlib:Using Keras backend: TENSORFLOW


In [2]:
dataset = openml.datasets.get_dataset(179)
X, y, _, names = dataset.get_data(target=dataset.default_target_attribute)

INFO:openml.datasets.dataset:pickle write adult


In [3]:
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = imputer.fit_transform(X)

In [4]:
X_discretized = X_imputed.copy()
for col in X.columns:
    if X[col].dtype == 'category':
        le = LabelEncoder()
        X_discretized[:, X.columns.get_loc(col)] = le.fit_transform(X_discretized[:, X.columns.get_loc(col)])


In [5]:
adult = fairlib.DataFrame(X_discretized, columns=names)
adult['income'] = y
adult["income"] = adult["income"].apply(lambda x: x == ">50K").astype(int)
adult

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country,income
0,2,6,77516.0,9,13,4,0,1,4,1,1,0,2,38,0
1,3,5,83311.0,9,13,2,3,0,4,1,0,0,0,38,0
2,2,3,215646.0,11,9,0,5,1,4,1,0,0,2,38,0
3,3,3,234721.0,1,7,2,5,0,2,1,0,0,2,38,0
4,1,3,338409.0,9,13,2,9,5,2,0,0,0,2,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,2,3,215419.0,9,13,0,9,1,4,0,0,0,2,38,0
48838,4,3,321403.0,11,9,6,9,2,2,1,0,0,2,38,0
48839,2,3,374983.0,9,13,2,9,0,4,1,0,0,3,38,0
48840,2,3,83891.0,9,13,0,0,3,1,1,2,0,2,38,0


In [6]:
adult.targets = "income"
adult.sensitive = 'sex' # fauci currently supports only one sensitive attribute

In [7]:
model = Sequential()
model.add(Dense(64, activation='relu'))  # Input layer and first hidden layer
model.add(Dense(32, activation='relu'))  # Second hidden layer
model.add(Dense(1, activation='softmax'))  # Output layer for classification

In [9]:
inprocessing = Fauci(model, loss='categorical_crossentropy', regularizer='sp', optimizer='adam', metrics=['accuracy'])

In [11]:
inprocessing.fit(adult, converting_to_type=float, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20


/Users/gciatto/Work/Students/master-thesis-dizio-ay2324/.venv/lib/python3.11/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/Users/gciatto/Work/Students/master-thesis-dizio-ay2324/fairlib/inprocessing/fauci.py:20: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return f(y_true, y_pred)


ValueError: Target and sensitive columns must be numeric